In [ ]:
import pandas as pd

try:
    # 원본 CSV 파일
    books = pd.read_csv('../data/Books.csv', low_memory=False)
    ratings = pd.read_csv('../data/Ratings.csv', low_memory=False)
    users = pd.read_csv('../data/Users.csv', low_memory=False)
    print("Complete download 3 CSV file")
except FileNotFoundError:
    print("Error : ../data/ 폴더에 CSV 파일이 있는지 확인하세요.")
    books, ratings, users = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

In [ ]:
# 신규 고객 유저 데이터
new_users_data = {
    'User-ID': [300000, 300001, 300002],
    'Location': ['usa', 'georgia, usa', 'morrow, georgia, usa'],
    'Age': [30, 30, 30] 
}

new_users_df = pd.DataFrame(new_users_data)

# 기존 users DataFrame에 신규 고객 데이터를 추가
users = pd.concat([users, new_users_df], ignore_index=True) 

print(f"신규 고객 3명 삽입 완료. 총 유저 수: {len(users)}")

In [ ]:
# 데이터 병합

if not (ratings.empty or books.empty or users.empty):
    # 'Ratings'를 중심으로 'Books' 정보와 'Users' 정보를 병합
    df_merged = pd.merge(ratings, books, on='ISBN')
    df_final = pd.merge(df_merged, users, on='User-ID')
    print("Complete data merge")
else:
    print("Error: 데이터 병합 실패.")

In [ ]:
# NaN 및 Outlier 처리
import numpy as np

# Age (나이) 처리
# 비정상적인 값(5세 미만, 100세 초과)과 NaN을 중앙값으로 대체
# 유효한 나이 범위(5~100) 내의 데이터 중앙값을 계산
median_age = df_final[(df_final['Age'] > 5) & (df_final['Age'] < 100)]['Age'].median()

# 비정상적인 나이 값(5 미만, 100 초과)을 중앙값으로 대체
df_final['Age'] = np.where(df_final['Age'] > 100, median_age, df_final['Age'])
df_final['Age'] = np.where(df_final['Age'] < 5, median_age, df_final['Age'])

# NaN 값 중앙값으로 채우기
df_final['Age'].fillna(median_age, inplace=True)
df_final['Age'] = df_final['Age'].astype(int)

# Year-Of-Publication (출판 연도) 처리
# 0이거나 현재 연도(2025년 기준)를 초과하는 비정상적인 값을 처리
current_year = 2025 

# 데이터 로드 시 str로 인식된 Year-Of-Publication 다시 int로 변환
df_final['Year-Of-Publication'] = pd.to_numeric(
    df_final['Year-Of-Publication'], 
    errors='coerce'
)

# 유효한 연도 범위(1900년 이후 ~ 현재) 내의 데이터 중앙값을 계산
median_year = df_final[(df_final['Year-Of-Publication'] > 1900) & (df_final['Year-Of-Publication'] <= current_year)]['Year-Of-Publication'].median()

# 0과 현재 연도를 초과하는 연도를 중앙값으로 대체
df_final['Year-Of-Publication'] = np.where(df_final['Year-Of-Publication'] > current_year, median_year, df_final['Year-Of-Publication'])
df_final['Year-Of-Publication'] = np.where(df_final['Year-Of-Publication'] == 0, median_year, df_final['Year-Of-Publication'])

# 처리되지 않은 NaN 값을 중앙값으로 대체
df_final['Year-Of-Publication'].fillna(median_year, inplace=True)

df_final['Year-Of-Publication'] = df_final['Year-Of-Publication'].astype(int)

print("Age 및 Year-Of-Publication 결측치/이상치 처리 완료.")

In [ ]:
## 노이즈 유저 및 아이템 제거 (데이터 품질 향상)

## 최소 상호작용 횟수
#MIN_USER_INTERACTIONS = 3
#MIN_ITEM_INTERACTIONS = 1

#print(f"--- 데이터 필터링 시작 (User 최소 {MIN_USER_INTERACTIONS}회, Item 최소 {MIN_ITEM_INTERACTIONS}회) ---")
#initial_rows = len(df_final)

## 노이즈 유저 제거 (상호작용 3회 미만)
#user_counts = df_final['User-ID'].value_counts()
#valid_users = user_counts[user_counts >= MIN_USER_INTERACTIONS].index
#df_final = df_final[df_final['User-ID'].isin(valid_users)]
#print(f"-> 유저 제거 후 상호작용: {len(df_final)}개")

## 노이즈 아이템 제거 (상호작용 1회 미만)
#item_counts = df_final['ISBN'].value_counts()
#valid_items = item_counts[item_counts >= MIN_ITEM_INTERACTIONS].index
#df_final = df_final[df_final['ISBN'].isin(valid_items)]
#print(f"-> 아이템 제거 후 최종 상호작용: {len(df_final)}개")

#final_rows = len(df_final)
#print(f"--- 필터링 완료: {initial_rows}개에서 {final_rows}개로 감소 ({100*(1 - final_rows/initial_rows):.2f}% 감소) ---")

In [ ]:
import numpy as np
import pickle
import re
import nltk
nltk.download('punkt')

from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelBinarizer
from lightfm.data import Dataset
from scipy.sparse import hstack, csr_matrix

# PorterStemmer 초기화
stemmer = PorterStemmer()

# text 데이터에 Stemming과 토큰화를 적용하는 함수
def stemming_tokenizer(text):
    if not isinstance(text, str):
        return []
    # 특수문자 및 숫자를 제거 (NLP 정제)
    text = re.sub(r'[^a-zA-Z\s]', '', text) 
    # 소문자 변환 후 토큰화 및 Stemming 적용
    return [stemmer.stem(w) for w in text.lower().split() if w]

# Location feature 정제 및 결합
def safe_split(location_str):
    if not isinstance(location_str, str): 
        return pd.Series(['', ''])
    parts = location_str.split(', ')
    # 국가와 시/도/군을 분리
    country = parts[-1] if len(parts) > 0 and parts[-1] else ''
    city_state = parts[0] if len(parts) > 0 and parts[0] else ''
    return pd.Series([city_state, country])

# 데이터 정제 및 특징 결합
df_final['Location'] = df_final['Location'].fillna('')
df_final[['Region', 'Country']] = df_final['Location'].apply(safe_split)
df_final['location_content'] = (df_final['Region'].fillna('') + ' ' + df_final['Country'].fillna('')).astype(str)

# 책 내용 특징 결합
fill_cols = ['Book-Title', 'Book-Author', 'Publisher']
df_final[fill_cols] = df_final[fill_cols].fillna('')
df_final['content'] = df_final['Book-Title'] + ' ' + df_final['Book-Author'] + ' ' + df_final['Publisher']

print("Complete preprocessing steps")

In [ ]:
from sklearn.preprocessing import LabelEncoder

# ID 매핑을 위한 인코더 생성
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

# 신규 고객 ID를 포함
user_encoder.fit(users['User-ID'].astype(str))

df_final['user_id_mapped'] = user_encoder.transform(df_final['User-ID'].astype(str))
df_final['item_id_mapped'] = item_encoder.fit_transform(df_final['ISBN'].astype(str))



# 임시 CountVectorizer 객체를 생성하여 feature 이름 목록을 가져옴
temp_location_vec = CountVectorizer(tokenizer=stemming_tokenizer)
temp_location_vec.fit(df_final['location_content'])
location_feature_names = list(temp_location_vec.get_feature_names_out())

temp_content_vec = CountVectorizer(tokenizer=stemming_tokenizer)
temp_content_vec.fit(df_final['content'])
content_feature_names = list(temp_content_vec.get_feature_names_out())

# Dataset Fit (정제된 feature 이름 등록)
dataset = Dataset()
dataset.fit(
    users=df_final['user_id_mapped'],
    items=df_final['item_id_mapped'],
    # Age (Label Binarization)와 Location (CountVectorizer) feature 이름 등록
    user_features=['Age'] + location_feature_names, 
    # Item Content (CountVectorizer)와 Year-Of-Publication (Label Binarization) feature 이름 등록
    item_features=content_feature_names + ['Year-Of-Publication']
)
print("Complete LightFM Dataset Fit")

In [ ]:
# 상호작용(Interactions) 매트릭스 생성

# (유저ID, 아이템ID, 평점) 튜플 리스트를 생성
interactions_data = list(zip(
    df_final['user_id_mapped'], 
    df_final['item_id_mapped'], 
    df_final['Book-Rating'].values
))

# dataset을 사용해 '상호작용' 행렬 생성
# interactions: 유저x아이템 행렬 (값이 1이면 상호작용, 0이면 X)
# weights: 유저x아이템 행렬 (값이 0~10점인 '평점' 행렬)
(interactions, weights) = dataset.build_interactions(interactions_data)

print("Complete Interactions Matrix")

In [ ]:
# LightFM용 피처 매트릭스 생성 (User / Item 분리)

# 중복 제거 및 인덱스 정렬
df_users = df_final.drop_duplicates(subset='user_id_mapped').set_index('user_id_mapped')
df_users = df_users.reindex(range(len(user_encoder.classes_))) 
df_items = df_final.drop_duplicates(subset='item_id_mapped').set_index('item_id_mapped')
df_items = df_items.reindex(range(len(item_encoder.classes_))) 

# User Features (Age, Location)

# Age: Label Binarizer (one-hot)
age_binarizer = LabelBinarizer(sparse_output=True)
user_age_features = age_binarizer.fit_transform(df_users['Age'].astype(str).values) 

# Location: Stemming + CountVectorizer
location_count_vec = CountVectorizer(tokenizer=stemming_tokenizer)
location_count_vec.fit(df_final['location_content']) 
location_count_matrix = location_count_vec.transform(df_users['location_content'].fillna(''))

# Age와 Location 피처를 hstack으로 결합
user_features = hstack([user_age_features, location_count_matrix]).tocsr()
print(f"Complete create User feature: {user_features.shape}") 

# Item Features (Content, Year)

# Book Content (Title, Author, Publisher): Stemming + CountVectorizer
content_count_vec = CountVectorizer(tokenizer=stemming_tokenizer)
content_count_matrix = content_count_vec.fit_transform(df_items['content'])

# Year-Of-Publication: Label Binarizer (one-hot)
year_binarizer = LabelBinarizer(sparse_output=True)
item_year_features = year_binarizer.fit_transform(df_items['Year-Of-Publication'].astype(str).values)

# Item Features 결합
item_features = hstack([content_count_matrix, item_year_features]).tocsr()
print(f"Complete create Item feature: {item_features.shape}")

In [ ]:
# 파일 저장
from scipy.sparse import hstack, csr_matrix, save_npz

save_npz('../data/interactions.npz', interactions)
save_npz('../data/weights.npz', weights)
# Age와 Location이 결합된 전체 user_features를 저장
save_npz('../data/user_features.npz', user_features) 
save_npz('../data/item_features.npz', item_features)

# 나중에 예측할 때 ID를 변환하기 위해 인코더(Encoder)도 저장
with open('../data/encoders.pkl', 'wb') as f:
    pickle.dump({'user_encoder': user_encoder, 'item_encoder': item_encoder}, f)

print("../data/interactions.npz")
print("../data/weights.npz")
print("../data/user_features.npz")
print("../data/item_features.npz")
print("../data/encoders.pkl")